In [2]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon, LineString, mapping, box, multilinestring


%matplotlib inline

# Wild and Scenic Designations – CA



In [ ]:
# Data file of CA polygons. 
ca_polys = gpd.read_file("wspolys/CaliforniaDesertProtectionAct_WSRs/")
ca_polys = ca_polys.to_crs(crs = nhd_1806.crs)

**Which NHD do we need?**

In [514]:
HUC4_subregions = gpd.read_file("NHD/WBD18_HU2_Shape/WBDHU4.shp") # NHD HUC4 boundaries

In [ ]:
# assign HUC4 codes to polygons
ca_polys_huc4 = gpd.sjoin(ca_polys, HUC4_subregions, how='left', op='intersects')

In [517]:
ca_polys_huc4.HUC4.unique()

array(['1807', '1810', nan, '1809', '1806'], dtype=object)

In [518]:
!ls NHD/

1806_Shape           1809_Shape           WBD18_HU2_Shape
1807_Shape           1810_Shape           WBD_18_HU2_Shape.zip


In [520]:
# load corresponding NHD flowlines for HUC4 code. 

nhd_1806 = gpd.read_file('NHD/1806_Shape/NHDFlowline.shp')

nhd_1807 = gpd.read_file('NHD/1807_Shape/NHDFlowline.shp')

nhd_1809 = gpd.read_file('NHD/1809_Shape/NHDFlowline.shp')

nhd_1810 = gpd.read_file('NHD/1810_Shape/NHDFlowline.shp')

# nhd is NSD83, need to convert
ca_polys = ca_polys.to_crs(crs = nhd_1806.crs)

In [521]:
# get intersecting polygons 

ca_polys_1807 = gpd.sjoin(nhd_1807, ca_polys, how='left', op='intersects')
ca_polys_1806 = gpd.sjoin(nhd_1806, ca_polys, how='left', op='intersects')
ca_polys_1809 = gpd.sjoin(nhd_1809, ca_polys, how='left', op='intersects')
ca_polys_1810 = gpd.sjoin(nhd_1810, ca_polys, how='left', op='intersects')


In [522]:
# filter on those with matches 

ca_polys_1807 = ca_polys_1807[ca_polys_1807.index_right.notnull()]
ca_polys_1806 = ca_polys_1806[ca_polys_1806.index_right.notnull()]
ca_polys_1809 = ca_polys_1809[ca_polys_1809.index_right.notnull()]
ca_polys_1810 = ca_polys_1810[ca_polys_1810.index_right.notnull()]

In [523]:
# merge all results. Each NHD polygon has a CA river designation polygon assigned to it, or none. 

all_06_07 = gpd.GeoDataFrame(pd.concat([ca_polys_1806, ca_polys_1807, ca_polys_1809, ca_polys_1810]), geometry = 'geometry')
all_06_07 = all_06_07[all_06_07.GNIS_Name.notnull()]

In [524]:
# group the polygons by ca-polygon River name and designation status

groups = all_06_07.groupby(['River', "Status"]).apply(lambda g: g[g.GNIS_Name.str.contains(g.name[0])])

In [527]:
# we had an issue with non-protected but intersectin tributaries. 
# We remove those flowlines which intersect with the designation boundary , by river name 
groups = all_06_07.groupby(['River', 'Status']).apply(lambda g: g[~g.intersects(ca_polys[ca_polys.River == g.name[0]].
                                                                                         cascaded_union.
                                                                                         boundary)])

In [528]:
groups = groups.drop(columns=['River', 'Status']).reset_index().dissolve(by=['River', 'Status'])
geoms = groups.geometry.values
# convert to multilinestrings for GeoJSON output.
geoms = [multilinestring.MultiLineString([g]) if g.type == "LineString" else g for g in geoms]

In [529]:
groups.geometry = geoms

In [539]:
groups = groups.reset_index()
groups.to_file("merged-all.geojson", driver='GeoJSON')


In [475]:
groups = all_06_07.groupby(['River', 'Status'])
groups.geometry = all_boundary_geoms

In [541]:
len(ca_polys.River.unique())

46

In [544]:
ca_polys_huc4[ca_polys_huc4.HUC4.isnull()]

,Status,River,Dedicated,Shape_Leng_left,Shape_Area_left,geometry,index_right,OBJECTID,TNMID,MetaSource,...,SourceFeat,LoadDate,AreaSqKm,AreaAcres,GNIS_ID,Name,States,HUC4,Shape_Leng_right,Shape_Area_right
13,Wild,Cottonwood Creek,None,57434.707225,1.412804e+07,POLYGON ((-118.2110044331431 37.56933608262781...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,Scenic,Cottonwood Creek,None,11920.160763,2.622635e+06,POLYGON ((-117.9949420950872 37.45621250339676...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
